In [2]:
import numpy as np, pandas as pd, os, pickle
import plotly.graph_objects as go, json
import plotly.express as px
from functools import reduce
from plotly.subplots import make_subplots
from glob import glob

In [3]:
databasepath = '/home/suraj/Desktop/rlalgorithms/database/assetmarket/'
resultpath = '/home/suraj/Desktop/rlalgorithms/results_H730_W003/metadata/'
dataname = 'PAM_FS2'
stockfile = 'foreignstock2.pkl'
datapath = os.path.join(resultpath, dataname)

assetnames = pickle.load(open(os.path.join(databasepath, stockfile), 'rb'))[2]
algos = [algo for algo in os.listdir(datapath) if '.html' not in algo]
algodata = []
for algo in algos:
    filepath = glob(os.path.join(datapath, algo,'*/*.csv')).pop()
    filedata = pd.read_csv(filepath, parse_dates=['timestamp'])
    filedata['algoname'] = algo
    algodata.append(filedata)

algodata = pd.concat(algodata, axis=0)
figs = [None]*2

In [4]:
transformname = {'timestamp':'Day', 'portfolio_value':'Portfolio value', 'algoname': 'Algorithm'}
figs[0] = px.line(algodata, x="timestamp", y="portfolio_value", color='algoname', title='Various PO Strategies', labels=transformname)

In [5]:
paframe = algodata[['timestamp', 'algoname', 'weights',]].copy().reset_index(drop=True)
paframe = paframe.loc[paframe.timestamp.dt.day==paframe.timestamp.dt.day[0], :].reset_index(drop=True)
paframe[f'weight_CASH'] = paframe['weights'].apply(lambda x: json.loads(x)[0])
for i, name in enumerate(assetnames):
    paframe[f'weight_{name}'] = paframe['weights'].apply(lambda x: json.loads(x)[i+1])
paframe.drop(columns=['weights'], inplace=True)
paframe = paframe.melt(id_vars=['timestamp', 'algoname'], value_vars=[name for name in paframe.columns if 'weight_' in name], var_name='asset', value_name='weight')
paframe.sort_values(['timestamp', 'algoname'], inplace=True)

paframe['timestamp'] = paframe['timestamp'].astype(str)
paframe['asset'] = paframe['asset'].apply(lambda x: x.replace('weight_', ''))
figs[1] = px.bar(paframe, x="asset", y="weight", color="algoname", barmode='group', animation_frame="timestamp", range_y=(0,1), title='Allocation over time')

In [6]:
with open(f'{resultpath}/{dataname}.html', 'w') as f:
    for fig in figs:
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))